In [ ]:
# Initialization
import csv
import os

In [ ]:
# Prompt the user to input the path to the CSV file
data_path = input("Enter the path to the data.csv file: ")

In [ ]:
# Function to create a dictionary for an interval with xmin, xmax, and text
def create_interval_data_dict(xmin, xmax, sentence):
    return {'xmin': float(xmin), 'xmax': float(xmax), 'text': sentence}

# Function to print interval data in a formatted way (for debugging, not used in this script)
def print_interval_data_dict(idx, data):
    print(f"intervals [{idx}]:\n\t\txmin = {data['xmin']}\n\t\txmax = {data['xmax']}\n\t\ttext = \"{data['text']}\"")

# Function to write intervals to a TextGrid file
def write_textgrid_file(intervals, output_file_path, total_xmax):
    print(f'Writing {output_file_path}')
    # Open the output file in write mode
    with open(output_file_path, 'w') as f:
        # Write the TextGrid file header
        f.write('File type = "ooTextFile"\n')
        f.write('Object class = "TextGrid"\n')
        f.write('\n')
        # Set the overall time boundaries (xmin = 0, xmax = total_xmax + 0.001 for buffer)
        f.write('xmin = 0\n')
        f.write(f'xmax = {str(float(total_xmax) + 0.001)}\n')
        f.write('tiers? <exists>\n')  # Indicate that tiers exist
        f.write('size = 1\n')       # Specify one tier
        f.write('item []:\n')       # Start the item section
        f.write('    item [1]:\n')  # Define the first (and only) tier
        f.write('        class = "IntervalTier"\n')  # Tier type
        f.write('        name = "Intonational Unit"\n')  # Name of the tier
        f.write('        xmin = 0\n')  # Tier starts at 0
        f.write(f'        xmax = {str(float(total_xmax) + 0.001)}\n')  # Tier ends at total_xmax + buffer
        # Number of intervals includes all defined intervals plus a final empty one
        f.write(f'        intervals: size = {len(intervals) + 1}\n')

        # Write each interval from the intervals list
        for idx, interval in enumerate(intervals):
            f.write(f'        intervals [{idx + 1}]:\n')  # Interval index starts at 1
            f.write(f'            xmin = {interval["xmin"]}\n')  # Start time
            f.write(f'            xmax = {interval["xmax"]}\n')  # End time
            f.write(f'            text = "{interval["text"]}"\n')  # Text content

        # Add a final empty interval if there are any intervals
        if len(intervals) > 0:
            f.write(f'        intervals [{len(intervals) + 1}]:\n')  # Final interval index
            f.write(f'            xmin = {intervals[-1]["xmax"]}\n')  # Starts at the last interval's end
            final_boundary = str(float(intervals[-1]["xmax"]) + 0.001)  # Ends slightly after last interval
            f.write(f'            xmax = {final_boundary}\n')
            f.write(f'            text = ""\n')  # Empty text for the final interval

# Define the output directory for TextGrid files
output_directory = 'textgrids'
# Create the directory if it doesn't exist, without raising an error if it already exists
os.makedirs(output_directory, exist_ok=True)

# Open the CSV file specified by the user
with open(data_path) as csvfile:
    reader = csv.reader(csvfile)  # Create a CSV reader object
    next(reader)  # Skip the header row (assumes first row is column names)

    # Initialize variables for tracking intonation unit (IU) boundaries and data
    iu_xmin = 0  # Start time of the current intonation unit
    iu_xmax = 0  # End time of the current intonation unit
    intervals = []  # List to store completed IU intervals
    words = []  # List to accumulate words within an IU
    prev_filename = None  # Track the previous filename to detect changes

    # Process each row in the CSV file
    for row in reader:
        try:
            # Extract data from the row
            filename = row[1]  # File name (e.g., VM21B_English_I1_20190313)
            xmin = float(row[2])  # Start time of the current word
            xmax = float(row[3])  # End time of the current word
            text = row[4]  # Word or text segment
            # Check if this is the start of an intonation unit (accepts both "True" and "TRUE")
            is_iu_start_pred = row[5] == "True" or row[5] == "TRUE"
        except IndexError:
            # Handle malformed rows (e.g., missing columns) by skipping them
            print(f"Skipping malformed row: {row}")
            continue

        # Check if the filename has changed (indicating a new file to process)
        if prev_filename is not None and prev_filename != filename:
            # If there are intervals from the previous file, write them to a TextGrid file
            if len(intervals) > 0:
                write_textgrid_file(intervals, os.path.join(output_directory, f"{prev_filename}.TextGrid"), intervals[-1]['xmax'])
            # Reset variables for the new file
            iu_xmin = 0
            iu_xmax = 0
            intervals = []
            words = []

        # Update the previous filename to the current one
        prev_filename = filename

        # If this row marks the start of a new intonation unit
        if is_iu_start_pred:
            # Determine the previous end time (0 if no intervals yet, otherwise last interval's xmax)
            if len(intervals) == 0:
                prev_xmax = 0
            else:
                prev_xmax = intervals[-1]['xmax']

            # If there's a gap between the previous end and the current IU start, add an empty interval
            if prev_xmax != iu_xmin:
                interval_data_dict = create_interval_data_dict(prev_xmax, iu_xmin, '')
                intervals.append(interval_data_dict)

            # If there are accumulated words, create an interval for the previous IU
            if len(words) > 0:
                interval_data_dict = create_interval_data_dict(iu_xmin, iu_xmax, ' '.join(words))
                intervals.append(interval_data_dict)
                words = []  # Clear the words list for the new IU

            # Set the start of the new IU to the current xmin
            iu_xmin = xmin

        # Add the current word to the words list and update the IU end time
        words.append(text)
        iu_xmax = xmax

    # After the loop, write the intervals for the last file if any exist
    if len(intervals) > 0:
        write_textgrid_file(intervals, os.path.join(output_directory, f"{prev_filename}.TextGrid"), intervals[-1]['xmax'])

Writing textgrids/VM21B_English_I1_20190313.TextGrid
Writing textgrids/VF19B_English_I1_20190213.TextGrid
Writing textgrids/VF27A_English_I1_20181120.TextGrid
Writing textgrids/VF22A_English_I2_20181206.TextGrid
Writing textgrids/VM19D_English_I2_20200211.TextGrid
Writing textgrids/VF19D_English_I2_20190308.TextGrid
Writing textgrids/VM21E_English_I2_20200309.TextGrid
Writing textgrids/VF23C_English_I2_20190128.TextGrid
Writing textgrids/VM22B_English_I1_20200309.TextGrid
Writing textgrids/VM21D_English_I2_20200309.TextGrid
Writing textgrids/VF33B_English_I1_20190206.TextGrid
Writing textgrids/VF21B_English_I2_20190204.TextGrid
Writing textgrids/VM19A_English_I1_20191031.TextGrid
Writing textgrids/VF23B_English_I1_20190121.TextGrid
Writing textgrids/VM22A_English_I2_20181210.TextGrid
Writing textgrids/VF20A_English_I2_20181119.TextGrid
Writing textgrids/VM21C_English_I2_20190403.TextGrid
Writing textgrids/VF32A_English_I2_20190213.TextGrid
Writing textgrids/VF21A_English_I1_20190130.Te